In [11]:
#import libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
### display image ###
def display(img1):
   plt.imshow(img1, cmap='gray')
   plt.axis('off')  # Turn off axis labels and ticks
   plt.show()

### Processing steps

##### Denoising (Adaptive non-local means)

In [7]:
###### adaptive NLM #####
def denoise(img, h=10, window=100, patch=7):
    result = cv2.fastNlMeansDenoising(img, None, h=h, templateWindowSize=patch, searchWindowSize=window)

    return result

##### Contrast enhancement (custom)

In [9]:
############# contrast enhancement ##################
def contrast_enhancement(img, threshold=30, box=(5,5)):
    # shape of input image
    rows, cols = img.shape

    #the result array  #same shape as input
    result = img.copy()

    # get width and height of the patch (box)
    box_width, box_height = box[1], box[0]

    # denoise the image one patch at a time
    for y in range(0, rows, box_height):
        for x in range(0, cols, box_width):
            # Extract the patch
            patch = img[y:y+box_height, x:x+box_width]

            # Calculate the average patch value
            avg = np.mean(patch[:,:])

            # if the patch average is considered black
            if avg <= threshold:
                # make the black pixels blacker
                patch[patch <= threshold] = patch[patch <= threshold]*(0.8)
                # make the non-black pixels brighter
                patch[patch > threshold] = avg*(1.2)
            #else:
                # make all pixels avg
                #patch[:, :] = avg*(1.1)

            # Update the patch in the result image
            result[y:y+box_height, x:x+box_width] = patch

    return result

##### Blurring (Gaussian blurr)

In [5]:
#Gaussian blurr
def blurr(img, kernel=(7,7)):
    denoised_image = cv2.GaussianBlur(img, kernel, 0) #apply Gaus. blur
    return denoised_image

##### Processing 

For black patch: make black pixels darker than the patch avg, and the non-black pixels brighter than avg.
For non-black patch : make all pixels slightly brighter than the patch average.


In [6]:
def process(img, threshold=25, box=(3,5)):
    # shape of input image
    rows, cols = img.shape

    #the result array  #same shape as input
    result = np.zeros((rows, cols))

    # get width and height of the patch (box)
    box_width, box_height = box[1], box[0]

    # process the image one patch at a time
    for y in range(0, rows, box_height):
        for x in range(0, cols, box_width):
            # Extract the patch
            patch = img[y:y+box_height, x:x+box_width]

            # Calculate the average patch value
            avg = np.mean(patch[:,:])

            # if the patch average is considered black
            if avg <= threshold:
                # make the black pixels blacker
                patch[patch <= threshold] = avg*(0.75)
                # make the non-black pixels brighter
                patch[patch > threshold] = avg*(1.5)
            else:
                # make all pixels avg
                patch[:, :] = avg*(1.1)

            # Update the patch in the result image
            result[y:y+box_height, x:x+box_width] = patch

    return result

### Processing functions

##### Process 1 image

In [10]:
def process_image(img):
    temp = denoise(img)
    temp = process(temp, threshold=25, box_size=(3,5))
    temp = blurr(temp, (5,5))
    result_image = contrast_enhancement(temp, threshold=30, box=(5,5))

    return result_image

##### Process a dataset

In [ ]:
# dataset = path of dataset
# dest_path = where output will be saved
def process_data(dataset, dest_path):
    # if dest folder doesnt exist, create it
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)

    # List of all files in the dataset
    image_files = os.listdir(dataset) #name of img file
    tot = len(image_files) 

    count = 0
    for image_file in image_files:
        # full path of the img = dataset path + image file name
        image_path = os.path.join(dataset, image_file)

        # Read the image as numpy array
        image = cv2.imread(image_path)

        # Process the image
        output_img = process_image(image)

        # full path of output = dest folder path + image file name
        output_path = os.path.join(dest_path, image_file)

        # Save the image
        cv2.imwrite(output_path, output_img)

        count+=1
        print(f"{count}/{tot}")
    print("All images processed succesfully.\n")


### Testing

In [ ]:
dataset = "input_images_folder"
dest_path = ".\\output" 

process_data(dataset, dest_path)